In [2]:
import requests
import warnings 
from urllib.request import urlopen
import time 
from bs4 import BeautifulSoup
import re
from  tqdm import tqdm
from datetime import datetime
import re
import pandas as pd

# 1

## 1.1. Ottenenere la lista dei link dei luoghi

In [3]:
exception=[]  #In questa lista vengono collocate le pagine web che danno problemi, se ci sono.


#Dal momento che le pagine web contengono dei numeri utilizziamo dei cicli for per cambiare il link delle pagine.

for i in tqdm(range(1,401)):
    url='https://www.atlasobscura.com/places?page='+str(i)+'&sort=likes_count' #Muoversi alla pagina successiva
    risposta=requests.get(url)                                                 #Richiedere  protocollo    https    
    testo=BeautifulSoup(risposta.text,'html.parser')                           #Formattare HTML
    
    try:
        first_div=testo.find_all("div",{"class":"row vue-js-bte-place-parent js-vue-component-wrap"}) #Trovare la tag
        
        
        for j in range(0,len(first_div)):
            try:
                second_div=first_div[j].find_all("div",{"class":"col-md-4 col-sm-6 col-xs-12"})
    
                for k in range(0,len(second_div)):
                    try:
                        a=second_div[k].find("a").get("href").strip()    #Estrazione del link della pagina
                     
                        with open('links.txt',"a")as f:
                            f.write("https://www.atlasobscura.com"+str(a))  #Scrivere su un file txt il link della pagina
                            f.write('\n')
                    except:
                        exception.append([i,j,k,risposta])      
            except:
                exception.append([i,j,None,risposta])   
    except:
        exception.append([i,None,None,risposta])
        
    
    time.sleep(1) #In ordere to avoid "429 Too Many Requests"
       
            
        

100%|█████████████████████████████████████████| 400/400 [12:46<00:00,  1.92s/it]


## 1.2. Crawl places

In [29]:
file = open('links.txt')
content = file.readlines()
for i in tqdm(range(0,len(content))):
    url=content[i].strip()    
    risposta=requests.get(url)                                               
    with open('HTML/html'+str(i)+'.txt',"a")as f:
        f.write(risposta.text)  #Scrivere sul file txt il link della sito del luogo.
    time.sleep(1)


100%|██████████| 7200/7200 [2:18:44<00:00,  1.16s/it]  


## 1.3 Parse downloaded pages

### Function that extract data from single HTML

In [28]:
def estrazione(testo):  
    errors=[]
    #1 Nome del luogo
    try:
        placeName=testo.find("h1",{"class":"DDPage__header-title"}).text.strip()
    except:
        placeName="Null"
        errors.append(1)
    #---#
    #2 Tag del luogo
    try:
        placeTags=[]
        ff2=testo.find("div",{"class":"item-tags col-xs-12"})
        ff2=ff2.find_all("span")
        for i in range(len(ff2)):
            placeTags.append(ff2[i].find("a").text.strip())
    except:
        placeTags=[" "]
        errors.append(2)



    #---#
    conta=testo.find_all("div",{"class","title-md item-action-count"})

    #3 Numero di persone che hanno visitato il luogo.
    try:
        numPeopleVisited=conta[0].text.strip()
    except:
        numPeopleVisited=" "
        errors.append(3)
    #---#
    #4   Numero di persone che vogliono visitare il luogo
    try:
        numPeopleWant=conta[1].text.strip()
    except:
        numPeopleWant=" "
        errors.append(4)
    #---#
    #5 Descrizione del luogo
    try:
        tag=testo.find("div",{"class":"DDP__body-copy"})
        tag=tag.find_all("p")
        placeDesc=""
        for i in range(0,len(tag)):
            placeDesc += str(tag[i].text)
            
        placeDesc=placeDesc.replace("  ","")    
    except:
        placeDesc=""
        errors.append(5)

    #6 Breve descrizione del luogo
    placeShortDesc=""
    try:
        placeShortDesc2=testo.find("h3", {"class","DDPage__header-dek"}).text.strip()
        
    except:
        placeShortDesc2=" "
        errors.append(6.2)
    placeShortDesc += placeShortDesc2
    
    placeShortDesc=placeShortDesc.replace("  ","")    
    #7 Luoghi vicini
    placeNearby=[]
    try:
        place=testo.find("div",{"class":"DDPageSiderail__column grid-col-lg-4 grid-col-md-5"})
        place=place.find("div",{"class":"DDPageSiderail"})
        place=place.find("div", {"class":"DDPageSiderailRecirc"})
        place=place.find_all("a")
        for i in range(0,len(place)):
            luogo=place[i].find("div",{"class":"DDPageSiderailRecirc__item-title"})
            placeNearby.append(luogo.text.strip())
    except:
        placeNearby=[" "]
        errors.append(7)
    #8 Indirizzo
    try:
        placeAddress=testo.find("div",{"class":"DDPageSiderail__column grid-col-lg-4 grid-col-md-5"})
        placeAddress=placeAddress.find("div",{"class":"DDPageSiderail"})
        placeAddress=placeAddress.find("aside", {"class":"DDPageSiderail__details"})
        placeAddress0=placeAddress.find("address",{"class":"DDPageSiderail__address"})
        
        placeAddress=placeAddress0.find("div").get_text(",").strip().rstrip(',').strip()
  
       
    except:
        placeAddress=""
        errors.append(8)
    
    try:
              
        placeAddress1=str(placeAddress0.find("div"))
        a=placeAddress1.replace("<div>","")
        a=a.replace("</div>","")
        indirizzo,citta,stato=a.split("<br/>")
    except:
        try:
            stato=placeAddress.split(",")
            stato=stato[len(stato)-1]
            indirizzo="Not available"
            citta="Not available"     
        except:
            indirizzo="Not available"
            citta="Not available"
            stato="Not available"
        
    
    #9 Latitudine e  longitudine
    try:
        coordinate=testo.find("div",{"class":"DDPageSiderail__column grid-col-lg-4 grid-col-md-5"})
        coordinate=coordinate.find("div",{"class":"DDPageSiderail"})
        coordinate=coordinate.find("aside", {"class":"DDPageSiderail__details"})
        coordinate=coordinate.find("div",{"class":"DDPageSiderail__coordinates js-copy-coordinates"}).text.strip().split()

        placeAlt=float(coordinate[0].replace(",",""))
        placeLong=float(coordinate[1].replace(",",""))
    except:
        placeAlt=" "
        placeLong=" "
        errors.append(9)

    #10 Nome degli editor e di chi contribuisce alla pagina dedicata al luogo.
    placeEditors=[]
    createdby=[]
    Editors=testo.find("div",{"id":"ugc-module"})
    Editors=Editors.find("div",{"class":"DDPContributors"})
    Editors=Editors.find_all("div",{"class":"ugc-editors"})

    try:
        for i in range(0,len(Editors)):
            tipo=Editors[i].find("h6").text.strip()
            if "edited by" == tipo.lower():
                try:
                    g=Editors[i].find("div",{"class":"DDPContributorsList"})
                    g=g.find("div",{"class":"js-editor-list hidden"})
                    g=g.find_all("li")
                    for  i in range(0,len(g)):
                        a=g[i].find("a").find("span").text.strip()
                        placeEditors.append(a)
                except:
                    placeEditors=[" "]
                    #errors.append(10.1)
                    
            elif "added by"==tipo.lower():
                try:
                    creator=Editors[i].find("div",{"class":"DDPContributorsList"})
                    createdby.append(creator.find("a").text)
                except:
                    createdby=[" "]
                   # errors.append(10.2)
    except:
        createdby=[" "]
        placeEditors=[" "] 
        errors.append(10)
    placeEditors=",".join(placeEditors)
    createdby=",".join(createdby)


    #11 Data di pubblicazione
    try:
        placePubDate=testo.find("div",{"id":"ugc-module"})
        placePubDate=placePubDate.find("div",{"class":"DDPContributors"})
        placePubDate=placePubDate.find("div",{"class":"DDPContributor__name"}).text.strip()
        placePubDate=datetime.strptime(placePubDate, "%B %d, %Y").date() #year, month, day
    except:
        placePubDate=" "
        errors.append(11)

    #12 Nome della lista in cui il luogo è compreso
    placeRelatedLists=[]
    try:
        o=testo.find_all("div",{"class":"card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links"})
        o=o[2].find_all("div",{"class":"CardWrapper js-CardWrapper"})
        for i in range(len(o)):
            m=o[i].find("h3").text.strip()
            placeRelatedLists.append(m)
    except:
        placeRelatedLists=[" "]
        errors.append(12)

    #13 Nome dei luoghi correlati
    placeRelatedPlaces=[]
    try:
        o=testo.find_all("div",{"class":"card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links"})
        o=o[1].find_all("div",{"class":"CardWrapper js-CardWrapper"})
        for i in range(len(o)):
            m=o[i].find("h3").text.strip()
            placeRelatedPlaces.append(m)
    except:
        placeRelatedPlaces=[" "]
        errors.append(13)

    #14 Link della luogo
    try:
        placeURL=testo.find("link",{"rel":"canonical"}).get("href")
    except:
        placeURL=" "
        errors.append(14)
    
    

    try:
        link=testo.find("div",{"class":"DDPage__header-place-location"})
        link=link.find("a").get("href").strip()
    except:
        link=" "
       
    
    d={"placeName":placeName,"placeTags": ",".join(placeTags),"numPeopleVisited":int(numPeopleVisited),"numPeopleWant":int(numPeopleWant),"placeDesc":placeDesc,"placeShortDesc":placeShortDesc,
    "placeNearby":",".join(placeNearby),"placeAddress":placeAddress.strip(),"placeAlt":placeAlt,"placeLong":placeLong,"createdby":createdby,"placeEditors":placeEditors,"placePubDate":placePubDate,
    "placeRelatedLists":",".join(placeRelatedLists),"placeRelatedPlaces":",".join(placeRelatedPlaces),"placeURL":placeURL}
    
    d_indirizzo={"placeName":placeName,"placeTags": ",".join(placeTags),"numPeopleVisited":int(numPeopleVisited),"numPeopleWant":int(numPeopleWant),"placeDesc":placeDesc,"placeShortDesc":placeShortDesc,
    "placeNearby":",".join(placeNearby),"placeAddress":placeAddress.strip(),"placeAlt":placeAlt,"placeLong":placeLong,"createdby":createdby,"placeEditors":placeEditors,"placePubDate":placePubDate,
    "placeRelatedLists":",".join(placeRelatedLists),"placeRelatedPlaces":",".join(placeRelatedPlaces),"placeURL":placeURL,"Address":indirizzo.strip(),"City":citta.strip(),"State":stato.strip()}
    
    return  d,errors,d_indirizzo    

## Create dataframe

#### Create 7200 tsv files, place_i.tsv and one csv file

In [29]:
anomalie={}
for i in tqdm(range(0,7200)):
    
    with open('HTML_scraping/html'+str(i)+'.txt') as f:
        testo = f.read()

    testo=BeautifulSoup(testo,'html.parser') 
    d,errori,d_indirizzo=estrazione(testo)
    
    if len(errori) != 0:
        anomalie[str(i)]=errori
        
    df=pd.DataFrame(d,index=[0])
   
    
    df.to_csv("TSV_data/place_"+str(i)+".tsv",sep="\t")
    if i==0 :
      dfff=pd.DataFrame(d,index=[i])
      dfff_indirizzo=pd.DataFrame(d_indirizzo,index=[0])
    
    else:
      dataframe=pd.DataFrame(d,index=[i])
      dfff=pd.concat([dfff,dataframe])
        
        
      dataframe_indirizzo=pd.DataFrame(d_indirizzo,index=[i])
      dfff_indirizzo=pd.concat([dfff_indirizzo,dataframe_indirizzo])
        
dfff.to_csv("places4.csv",sep="\t")
dfff_indirizzo.to_csv("address.csv",sep="\t")

100%|███████████████████████████████████████| 7200/7200 [10:45<00:00, 11.15it/s]


#### Overview of the dataset

In [30]:
df=pd.read_csv("places4.csv",delimiter="\t")
df.shape

(7200, 17)

In [31]:
df=pd.read_csv("address.csv",delimiter="\t")
df.shape

(7200, 20)

#### Analysis of the errors

In [41]:
with open('anomalie.txt',"w")as f:
    for i in anomalie.keys():
        f.write(i)
        f.write(str(anomalie[i]))
        f.write("\n")